In [54]:
from bs4 import BeautifulSoup as bs
import requests

url = "https://www.bbc.com/news/health-61553822"
response = requests.get(url)
#print(response)

html = response.content
#print(html)
soup = bs(html,"lxml")


In [67]:
import re
article = " ".join([el.text for el in soup.find_all("div", [re.compile('.*RichTextContainer.*')], recursive=True)])
#article = [el.text for el in soup.find_all("div", [re.compile('.*RichTextContainer.*')], recursive= True)]
print(article)

Monkeypox can be contained in countries outside of Africa where the virus is not usually detected, the World Health Organization (WHO) says. More than 100 cases of the virus - which causes a rash and a fever - have been confirmed in Europe, the Americas and Australia. That number is expected to rise still, but experts say the overall risk to the broader population is very low. The virus is most common in remote parts of Central and West Africa. "This is a containable situation," the WHO's emerging disease lead Maria Van Kerkhove said at a news conference on Monday. "We want to stop human-to-human transmission. We can do this in the non-endemic countries," she added - referring to recent cases in Europe and North America. What is monkeypox and how do you catch it? Monkeypox: Time to worry or one to ignore? The virus has now been detected in 16 countries outside Africa. Despite being the largest outbreak outside of Africa in 50 years, monkeypox does not spread easily between people and e

In [ ]:
#puntuation  removal
clean_article = re.sub(f"[{re.escape(punctuation)}]", "", article)
print(clean_article)

In [68]:
import spacy
import textwrap
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
punctuation += '\n' 
stopwords = list(STOP_WORDS)

reduction_rate = 0.1  #defines how small the output summary should be compared with the input 


In [69]:
nlp_pl = spacy.load('en_core_web_sm')     #process original text according with the Spacy nlp pipeline for english
document = nlp_pl(article)                   #doc object

tokens = [token.text for token in document] #tokenized text

word_frequencies = {}
for word in document:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

max_frequency = max(word_frequencies.values())
print(max_frequency)

#Normalisation of word_freq
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency

print(word_frequencies)

6
{'Monkeypox': 0.5, 'contained': 0.16666666666666666, 'countries': 0.6666666666666666, 'outside': 0.5, 'Africa': 0.6666666666666666, 'virus': 1.0, 'usually': 0.16666666666666666, 'detected': 0.3333333333333333, 'World': 0.16666666666666666, 'Health': 0.16666666666666666, 'Organization': 0.16666666666666666, 'says': 0.16666666666666666, '100': 0.16666666666666666, 'cases': 0.5, 'causes': 0.16666666666666666, 'rash': 0.3333333333333333, 'fever': 0.16666666666666666, 'confirmed': 0.3333333333333333, 'Europe': 0.3333333333333333, 'Americas': 0.16666666666666666, 'Australia': 0.16666666666666666, 'number': 0.16666666666666666, 'expected': 0.16666666666666666, 'rise': 0.16666666666666666, 'experts': 0.3333333333333333, 'overall': 0.16666666666666666, 'risk': 0.5, 'broader': 0.3333333333333333, 'population': 0.3333333333333333, 'low': 0.3333333333333333, 'common': 0.16666666666666666, 'remote': 0.16666666666666666, 'parts': 0.16666666666666666, 'Central': 0.16666666666666666, 'West': 0.16666

In [70]:
sentence_tokens = [sent for sent in document.sents]

def get_sentence_scores(sentence_tok, len_norm=True):
  sentence_scores = {}
  for sent in sentence_tok:
      word_count = 0
      for word in sent:
          if word.text.lower() in word_frequencies.keys():
              word_count += 1
              if sent not in sentence_scores.keys():
                  sentence_scores[sent] = word_frequencies[word.text.lower()]
              else:
                  sentence_scores[sent] += word_frequencies[word.text.lower()]
      if len_norm:
        sentence_scores[sent] = sentence_scores[sent]/word_count
  return sentence_scores
                
sentence_scores = get_sentence_scores(sentence_tokens,len_norm=False)        #sentence scoring without lenght normalization
sentence_scores_rel = get_sentence_scores(sentence_tokens,len_norm=True)     #sentence scoring with length normalization

In [71]:
def get_summary(sentence_sc, rate):
  summary_length = int(len(sentence_sc)*rate)
  summary = nlargest(summary_length, sentence_sc, key = sentence_sc.get)
  final_summary = [word.text for word in summary]
  summary = ' '.join(final_summary)
  return summary

print("- NON_REL: "+ get_summary(sentence_scores, reduction_rate))
print("- REL: "+ get_summary(sentence_scores_rel, reduction_rate))

- NON_REL: A person is considered at high risk of having caught the infection if they have had household or sexual contact with someone with monkeypox, or have changed the bedding of an infected person without wearing personal protective equipment (PPE).  "However the likelihood of further spread of the virus through close contact for example during sexual activities amongst persons with multiple sexual partners is considered to be high".
- REL: The virus has now been detected in 16 countries outside Africa. What is monkeypox and how do you catch it?
